# IBKR Current Positions - Vertical Spreads

This notebook retrieves current positions from Interactive Brokers and focuses on vertical spread positions with detailed information.

In [1]:
import time
import threading
import pandas as pd
import random
import datetime
from decimal import Decimal
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def safe_float_convert(value):
    """Safely convert any value to float, handling Decimal types"""
    if value is None or pd.isna(value):
        return None
    if isinstance(value, str):
        try:
            return float(value)
        except ValueError:
            return None
    if isinstance(value, Decimal):
        return float(value)
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

In [2]:
class IBKRPositionApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.positions = {}
        self.market_data = {}
        self.req_id = 1000
        self.position_data_received = False
        self.market_data_requests = {}
        
    def connectTWS(self, port=7497):
        """Connect to TWS or IB Gateway"""
        try:
            client_id = random.randint(1, 999)
            # Use Windows host IP for WSL connections
            import subprocess
            try:
                # Get Windows host IP from WSL - try gateway IP first
                result = subprocess.run(['ip', 'route', 'show'], capture_output=True, text=True)
                host_ip = None
                for line in result.stdout.split('\n'):
                    if 'default via' in line:
                        host_ip = line.split()[2]
                        break
                
                # Fallback to resolv.conf method
                if not host_ip:
                    result = subprocess.run(['cat', '/etc/resolv.conf'], capture_output=True, text=True)
                    for line in result.stdout.split('\n'):
                        if 'nameserver' in line:
                            host_ip = line.split()[-1]
                            break
                
                if not host_ip:
                    host_ip = '127.0.0.1'  # Final fallback
            except:
                host_ip = '127.0.0.1'  # Fallback
            
            print(f"Attempting to connect to {host_ip}:{port}")
            self.connect(host_ip, port, client_id)
            
            # Start message processing thread
            thread = threading.Thread(target=self.run)
            thread.daemon = True
            thread.start()
            
            time.sleep(2)  # Wait for connection
            
            if self.isConnected():
                logger.info(f"Connected to TWS/Gateway on {host_ip}:{port} with client ID {client_id}")
                return True
            else:
                logger.error("Failed to connect to TWS/Gateway")
                return False
                
        except Exception as e:
            logger.error(f"Connection error: {e}")
            return False
    
    def position(self, account, contract, position, avgCost):
        """Callback for position data"""
        key = f"{contract.symbol}_{contract.secType}_{contract.strike}_{contract.right}_{contract.lastTradeDateOrContractMonth}"
        
        # Convert all numeric values to float immediately
        avg_cost_float = safe_float_convert(avgCost)
        position_float = safe_float_convert(position)
        strike_float = safe_float_convert(contract.strike) if hasattr(contract, 'strike') else None
        
        self.positions[key] = {
            'Account': account,
            'Symbol': contract.symbol,
            'SecType': contract.secType,
            'Description': f"{contract.symbol} {contract.lastTradeDateOrContractMonth} {contract.strike} {contract.right}",
            'AvgCost': avg_cost_float,
            'Strike': strike_float,
            'Right': contract.right if hasattr(contract, 'right') else None,
            'Expiry': contract.lastTradeDateOrContractMonth if hasattr(contract, 'lastTradeDateOrContractMonth') else None,
            'Position': position_float,
            'Contract': contract,
            'CurrentPrice': None,
            'PriceSource': None,
            'Bid': None,
            'Ask': None,
            'UnrealizedPnL': None,
            'MarketVal': None
        }
        
        logger.info(f"Position: {key} - Qty: {position_float}")
    
    def positionEnd(self):
        """Callback when all positions have been received"""
        self.position_data_received = True
        logger.info("All position data received")
    
    def tickPrice(self, reqId, tickType, price, attrib):
        """Callback for market data price ticks"""
        if reqId in self.market_data_requests:
            key = self.market_data_requests[reqId]
            
            if key not in self.market_data:
                self.market_data[key] = {}
            
            # Convert price to float immediately
            price_float = safe_float_convert(price)
            
            # Debug logging
            symbol = key.split('_')[0]
            tick_name = {1: 'Bid', 2: 'Ask', 4: 'Last', 6: 'High', 7: 'Low', 9: 'Close'}.get(tickType, f"Type{tickType}")
            print(f"Market data: {symbol} - {tick_name}: ${price_float}")
            
            # tickType 1 = Bid, tickType 2 = Ask, tickType 4 = Last
            if tickType == 1:  # Bid
                self.market_data[key]['Bid'] = price_float
            elif tickType == 2:  # Ask
                self.market_data[key]['Ask'] = price_float
            elif tickType == 4:  # Last
                self.market_data[key]['Last'] = price_float
            elif tickType == 9:  # Close
                self.market_data[key]['Close'] = price_float

    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson="", *args):
        """Error callback - accepts variable arguments"""
        # Filter out common informational messages
        if errorCode not in [2104, 2106, 2158]:  # Market data farm connection messages
            print(f"Error: ID={reqId}, Code={errorCode}, Msg={errorString}")
        
    def tickSnapshotEnd(self, reqId):
        """Called when snapshot is complete"""
        if reqId in self.market_data_requests:
            key = self.market_data_requests[reqId]
            symbol = key.split('_')[0]
            print(f"Snapshot complete for {symbol} (ReqId: {reqId})")
    
    def get_positions_data(self):
        """Request positions and wait for data"""
        logger.info("Requesting positions...")
        self.reqPositions()
        
        # Wait for position data
        timeout = 30
        start_time = time.time()
        
        while not self.position_data_received and (time.time() - start_time) < timeout:
            time.sleep(0.1)
        
        if not self.position_data_received:
            logger.warning("Timeout waiting for position data")
        
        return len(self.positions)
    
    def get_market_data_for_positions(self):
        """Request market data for all positions"""
        logger.info("Requesting market data for positions...")
        print(f"Total positions to request market data for: {len(self.positions)}")
        
        for key, position_data in self.positions.items():
            contract = position_data['Contract']
            req_id = self.req_id
            self.market_data_requests[req_id] = key
            
            # Set exchange for options - THIS IS THE FIX
            if contract.secType == 'OPT':
                contract.exchange = 'SMART'  # Use SMART routing for options
                contract.primaryExchange = ''
            elif contract.secType == 'STK':
                contract.exchange = 'SMART'
            
            print(f"Requesting market data for {contract.symbol} on {contract.exchange} (ReqId: {req_id})")
            
            # Request snapshot market data
            self.reqMktData(req_id, contract, "", True, False, [])  # snapshot=True
            self.req_id += 1
            time.sleep(0.2)  # Delay between requests
        
        print(f"Waiting for market data responses...")
        time.sleep(15)  # Wait for responses
        
        # Cancel requests
        for req_id in list(self.market_data_requests.keys()):
            self.cancelMktData(req_id)
        
        print(f"Received market data for {len(self.market_data)} positions")
        
        # Update positions with market data
        updated_count = 0
        for key in self.positions.keys():
            if key in self.market_data:
                market_info = self.market_data[key]
                
                current_price = None
                price_source = None
                
                if 'Last' in market_info and market_info['Last'] and market_info['Last'] > 0:
                    current_price = market_info['Last']
                    price_source = 'Last'
                elif 'Close' in market_info and market_info['Close'] and market_info['Close'] > 0:
                    current_price = market_info['Close']
                    price_source = 'Close'
                elif 'Bid' in market_info and 'Ask' in market_info:
                    if market_info['Bid'] and market_info['Ask'] and market_info['Bid'] > 0 and market_info['Ask'] > 0:
                        current_price = (market_info['Bid'] + market_info['Ask']) / 2
                        price_source = 'Bid/Ask Mid'
                
                if current_price is not None:
                    updated_count += 1
                    
                self.positions[key]['CurrentPrice'] = current_price
                self.positions[key]['PriceSource'] = price_source
                self.positions[key]['Bid'] = market_info.get('Bid', None)
                self.positions[key]['Ask'] = market_info.get('Ask', None)
                
                # Calculate market value and unrealized PnL
                if current_price is not None:
                    position_qty = self.positions[key]['Position']
                    avg_cost = self.positions[key]['AvgCost']
                    
                    if position_qty is not None and avg_cost is not None:
                        multiplier = 100 if self.positions[key]['SecType'] == 'OPT' else 1
                        market_val = current_price * position_qty * multiplier
                        unrealized_pnl = (current_price - avg_cost) * position_qty * multiplier
                        
                        self.positions[key]['MarketVal'] = market_val
                        self.positions[key]['UnrealizedPnL'] = unrealized_pnl
            else:
                symbol = key.split('_')[0]
                print(f"No market data received for {symbol}")
        
        print(f"Successfully updated prices for {updated_count} positions")
    
    def get_positions_dataframe(self):
        """Convert positions to DataFrame with specified columns"""
        data = []
        
        for key, pos in self.positions.items():
            data.append({
                'Account': pos['Account'],
                'Symbol': pos['Symbol'],
                'SecType': pos['SecType'],
                'Description': pos['Description'],
                'AvgCost': pos['AvgCost'],
                'CurrentPrice': pos['CurrentPrice'],
                'PriceSource': pos['PriceSource'],
                'Bid': pos['Bid'],
                'Ask': pos['Ask'],
                'UnrealizedPnL': pos['UnrealizedPnL'],
                'Strike': pos['Strike'],
                'Right': pos['Right'],
                'Expiry': pos['Expiry'],
                'MarketVal': pos['MarketVal'],
                'Position': pos['Position']
            })
        
        return pd.DataFrame(data)
    
    def find_vertical_spreads(self, df):
        """Identify vertical spreads from options positions"""
        options_df = df[df['SecType'] == 'OPT'].copy()
        
        if options_df.empty:
            return pd.DataFrame()
        
        spreads = []
        
        for (symbol, expiry, right), group in options_df.groupby(['Symbol', 'Expiry', 'Right']):
            if len(group) >= 2:
                group = group.sort_values('Strike')
                
                for i in range(len(group)):
                    for j in range(i + 1, len(group)):
                        pos1 = group.iloc[i]
                        pos2 = group.iloc[j]
                        
                        strike1 = safe_float_convert(pos1['Strike'])
                        strike2 = safe_float_convert(pos2['Strike'])
                        avg_cost1 = safe_float_convert(pos1['AvgCost'])
                        avg_cost2 = safe_float_convert(pos2['AvgCost'])
                        position1 = safe_float_convert(pos1['Position'])
                        position2 = safe_float_convert(pos2['Position'])
                        
                        if any(v is None for v in [strike1, strike2, avg_cost1, avg_cost2, position1, position2]):
                            continue
                        
                        if (position1 > 0 and position2 < 0) or (position1 < 0 and position2 > 0):
                            if abs(position1) == abs(position2):
                                spread_type = "Bull" if (strike1 < strike2 and position1 > 0) else "Bear"
                                spread_type += " Call" if right == "C" else " Put"
                                
                                net_cost = (avg_cost1 * position1 + avg_cost2 * position2) / abs(position1)
                                current_value = None
                                
                                current1 = safe_float_convert(pos1['CurrentPrice'])
                                current2 = safe_float_convert(pos2['CurrentPrice'])
                                
                                if current1 is not None and current2 is not None:
                                    current_value = (current1 * position1 + current2 * position2) / abs(position1)
                                
                                spreads.append({
                                    'Account': pos1['Account'],
                                    'Symbol': symbol,
                                    'SecType': 'SPREAD',
                                    'Description': f"{spread_type} {strike1}/{strike2} {expiry}",
                                    'AvgCost': net_cost,
                                    'CurrentPrice': current_value,
                                    'PriceSource': 'Calculated',
                                    'Bid': None,
                                    'Ask': None,
                                    'UnrealizedPnL': (current_value - net_cost) * abs(position1) * 100 if current_value else None,
                                    'Strike': f"{strike1}/{strike2}",
                                    'Right': right,
                                    'Expiry': expiry,
                                    'MarketVal': current_value * abs(position1) * 100 if current_value else None,
                                    'Position': abs(position1)
                                })
        
        return pd.DataFrame(spreads)
    
    def disconnect_tws(self):
        """Disconnect from TWS"""
        if self.isConnected():
            self.disconnect()
            logger.info("Disconnected from TWS/Gateway")

In [ ]:
# Initialize the IBKR application
app = IBKRPositionApp()

# Connect to TWS/Gateway
connected = False
ports_to_try = [7497, 7496]  # Paper TWS, Live TWS

for port in ports_to_try:
    print(f"Attempting to connect on port {port}...")
    if app.connectTWS(port):
        connected = True
        print(f"Successfully connected on port {port}")
        break
    else:
        print(f"Failed to connect on port {port}")

if not connected:
    print("Could not connect to TWS/Gateway.")
    print("Make sure TWS is running and API is enabled with trusted IPs configured.")
else:
    print("Connected to IBKR successfully!")

Attempting to connect on port 7497...
Attempting to connect to 172.21.240.1:7497


INFO:ibapi.client:sent startApi
INFO:ibapi.client:REQUEST startApi {}
INFO:ibapi.client:SENDING startApi b'\x00\x00\x00\n71\x002\x00281\x00\x00'
INFO:ibapi.wrapper:ANSWER connectAck {}
INFO:ibapi.wrapper:ANSWER nextValidId {'orderId': 1}
INFO:ibapi.wrapper:ANSWER managedAccounts {'accountsList': 'DU9233079'}
INFO:__main__:Connected to TWS/Gateway on 172.21.240.1:7497 with client ID 281


Successfully connected on port 7497
Connected to IBKR successfully!


INFO:__main__:Position: TSM_OPT_215.0_P_20250912 - Qty: -1.0
INFO:__main__:Position: IWM_OPT_260.0_C_20250919 - Qty: 1.0
INFO:__main__:Position: TSM_OPT_205.0_P_20250912 - Qty: 1.0
INFO:__main__:Position: IWM_OPT_248.0_C_20250919 - Qty: -1.0
INFO:__main__:Position: UNH_OPT_350.0_C_20250926 - Qty: -1.0
INFO:__main__:Position: UNH_OPT_360.0_C_20250926 - Qty: 1.0
INFO:__main__:All position data received
INFO:ibapi.wrapper:ANSWER tickReqParams {'tickerId': 1000, 'minTick': 0.01, 'bboExchange': 'c70003', 'snapshotPermissions': 0}
INFO:ibapi.wrapper:ANSWER marketDataType {'reqId': 1000, 'marketDataType': 1}
INFO:ibapi.wrapper:ANSWER tickSize {'reqId': 1000, 'tickType': 0, 'size': 0}
INFO:ibapi.wrapper:ANSWER tickSize {'reqId': 1000, 'tickType': 3, 'size': 0}
INFO:ibapi.wrapper:ANSWER tickSize {'reqId': 1000, 'tickType': 8, 'size': 0}
INFO:ibapi.wrapper:ANSWER tickGeneric {'reqId': 1000, 'tickType': 49, 'value': 0.0}


Market data: TSM - Bid: $-1.0
Market data: TSM - Ask: $-1.0
Market data: TSM - Close: $0.0


...

Market data: IWM - Bid: $-1.0
Market data: IWM - Ask: $-1.0
Market data: IWM - Close: $0.65


INFO:ibapi.wrapper:ANSWER tickReqParams {'tickerId': 1004, 'minTick': 0.01, 'bboExchange': 'c70003', 'snapshotPermissions': 0}
INFO:ibapi.wrapper:ANSWER marketDataType {'reqId': 1004, 'marketDataType': 1}
INFO:ibapi.wrapper:ANSWER tickSize {'reqId': 1004, 'tickType': 0, 'size': 0}
INFO:ibapi.wrapper:ANSWER tickSize {'reqId': 1004, 'tickType': 3, 'size': 0}
INFO:ibapi.wrapper:ANSWER tickSize {'reqId': 1004, 'tickType': 8, 'size': 0}
INFO:ibapi.wrapper:ANSWER tickGeneric {'reqId': 1004, 'tickType': 49, 'value': 0.0}


Market data: UNH - Bid: $-1.0
Market data: UNH - Ask: $-1.0
Market data: UNH - Close: $11.02


INFO:ibapi.wrapper:ANSWER tickReqParams {'tickerId': 1005, 'minTick': 0.01, 'bboExchange': 'c70003', 'snapshotPermissions': 0}
INFO:ibapi.wrapper:ANSWER marketDataType {'reqId': 1005, 'marketDataType': 1}
INFO:ibapi.wrapper:ANSWER tickSize {'reqId': 1005, 'tickType': 0, 'size': 0}
INFO:ibapi.wrapper:ANSWER tickSize {'reqId': 1005, 'tickType': 3, 'size': 0}
INFO:ibapi.wrapper:ANSWER tickSize {'reqId': 1005, 'tickType': 8, 'size': 0}
INFO:ibapi.wrapper:ANSWER tickGeneric {'reqId': 1005, 'tickType': 49, 'value': 0.0}


Market data: UNH - Bid: $-1.0
Market data: UNH - Ask: $-1.0
Market data: UNH - Close: $6.57
Snapshot complete for TSM (ReqId: 1000)
Snapshot complete for IWM (ReqId: 1001)
Snapshot complete for TSM (ReqId: 1002)
Snapshot complete for IWM (ReqId: 1003)
Snapshot complete for UNH (ReqId: 1004)
Snapshot complete for UNH (ReqId: 1005)
Error: ID=1000, Code=300, Msg=Can't find EId with tickerId:1000
Error: ID=1001, Code=300, Msg=Can't find EId with tickerId:1001
Error: ID=1002, Code=300, Msg=Can't find EId with tickerId:1002
Error: ID=1003, Code=300, Msg=Can't find EId with tickerId:1003
Error: ID=1004, Code=300, Msg=Can't find EId with tickerId:1004
Error: ID=1005, Code=300, Msg=Can't find EId with tickerId:1005


In [4]:
# Get positions data
if connected:
    print("Requesting positions...")
    num_positions = app.get_positions_data()
    print(f"Retrieved {num_positions} positions")
    
    if num_positions > 0:
        print("Requesting market data for positions...")
        app.get_market_data_for_positions()
        print("Market data request completed")
    else:
        print("No positions found")

INFO:__main__:Requesting positions...
INFO:ibapi.client:REQUEST reqPositions {}
INFO:ibapi.client:SENDING reqPositions b'\x00\x00\x00\x0561\x001\x00'
INFO:__main__:Requesting market data for positions...
INFO:ibapi.client:REQUEST reqMktData {'reqId': 1000, 'contract': 140366411516144: 803395622,TSM,OPT,20250912,215.0,P,100,SMART,,USD,TSM   250912P00215000,TSM,False,,combo:, 'genericTickList': '', 'snapshot': True, 'regulatorySnapshot': False, 'mktDataOptions': []}
INFO:ibapi.client:SENDING reqMktData b'\x00\x00\x00^1\x0011\x001000\x00803395622\x00TSM\x00OPT\x0020250912\x00215.0\x00P\x00100\x00SMART\x00\x00USD\x00TSM   250912P00215000\x00TSM\x000\x00\x001\x000\x00\x00'


Requesting positions...
Retrieved 6 positions
Requesting market data for positions...
Total positions to request market data for: 6
Requesting market data for TSM on SMART (ReqId: 1000)


INFO:ibapi.client:REQUEST reqMktData {'reqId': 1001, 'contract': 140366411518160: 712424464,IWM,OPT,20250919,260.0,C,100,SMART,,USD,IWM   250919C00260000,IWM,False,,combo:, 'genericTickList': '', 'snapshot': True, 'regulatorySnapshot': False, 'mktDataOptions': []}
INFO:ibapi.client:SENDING reqMktData b'\x00\x00\x00^1\x0011\x001001\x00712424464\x00IWM\x00OPT\x0020250919\x00260.0\x00C\x00100\x00SMART\x00\x00USD\x00IWM   250919C00260000\x00IWM\x000\x00\x001\x000\x00\x00'


Requesting market data for IWM on SMART (ReqId: 1001)


INFO:ibapi.client:REQUEST reqMktData {'reqId': 1002, 'contract': 140366411518592: 803395492,TSM,OPT,20250912,205.0,P,100,SMART,,USD,TSM   250912P00205000,TSM,False,,combo:, 'genericTickList': '', 'snapshot': True, 'regulatorySnapshot': False, 'mktDataOptions': []}
INFO:ibapi.client:SENDING reqMktData b'\x00\x00\x00^1\x0011\x001002\x00803395492\x00TSM\x00OPT\x0020250912\x00205.0\x00P\x00100\x00SMART\x00\x00USD\x00TSM   250912P00205000\x00TSM\x000\x00\x001\x000\x00\x00'


...

Requesting market data for UNH on SMART (ReqId: 1004)


INFO:ibapi.client:REQUEST reqMktData {'reqId': 1005, 'contract': 140366411516576: 805456105,UNH,OPT,20250926,360.0,C,100,SMART,,USD,UNH   250926C00360000,UNH,False,,combo:, 'genericTickList': '', 'snapshot': True, 'regulatorySnapshot': False, 'mktDataOptions': []}
INFO:ibapi.client:SENDING reqMktData b'\x00\x00\x00^1\x0011\x001005\x00805456105\x00UNH\x00OPT\x0020250926\x00360.0\x00C\x00100\x00SMART\x00\x00USD\x00UNH   250926C00360000\x00UNH\x000\x00\x001\x000\x00\x00'


Requesting market data for UNH on SMART (ReqId: 1005)
Waiting for market data responses...


INFO:ibapi.client:REQUEST cancelMktData {'reqId': 1000}
INFO:ibapi.client:SENDING cancelMktData b'\x00\x00\x00\t2\x002\x001000\x00'
INFO:ibapi.client:REQUEST cancelMktData {'reqId': 1001}
INFO:ibapi.client:SENDING cancelMktData b'\x00\x00\x00\t2\x002\x001001\x00'
INFO:ibapi.client:REQUEST cancelMktData {'reqId': 1002}
INFO:ibapi.client:SENDING cancelMktData b'\x00\x00\x00\t2\x002\x001002\x00'
INFO:ibapi.client:REQUEST cancelMktData {'reqId': 1003}
INFO:ibapi.client:SENDING cancelMktData b'\x00\x00\x00\t2\x002\x001003\x00'
INFO:ibapi.client:REQUEST cancelMktData {'reqId': 1004}
INFO:ibapi.client:SENDING cancelMktData b'\x00\x00\x00\t2\x002\x001004\x00'
INFO:ibapi.client:REQUEST cancelMktData {'reqId': 1005}
INFO:ibapi.client:SENDING cancelMktData b'\x00\x00\x00\t2\x002\x001005\x00'


Received market data for 6 positions
Successfully updated prices for 4 positions
Market data request completed


In [5]:
# Create DataFrame with all positions
if connected and len(app.positions) > 0:
    positions_df = app.get_positions_dataframe()
    
    print(f"\nAll Positions ({len(positions_df)} total):")
    print("=" * 100)
    
    columns_order = ['Account', 'Symbol', 'SecType', 'Description', 'AvgCost', 
                    'CurrentPrice', 'PriceSource', 'Bid', 'Ask', 'UnrealizedPnL', 
                    'Strike', 'Right', 'Expiry', 'MarketVal', 'Position']
    
    display_df = positions_df[columns_order].copy()
    
    numeric_cols = ['AvgCost', 'CurrentPrice', 'Bid', 'Ask', 'UnrealizedPnL', 'MarketVal']
    for col in numeric_cols:
        display_df[col] = display_df[col].apply(lambda x: f"${x:.2f}" if pd.notnull(x) else "N/A")
    
    print(display_df.to_string(index=False))
else:
    print("No positions data available")


All Positions (6 total):
  Account Symbol SecType          Description AvgCost CurrentPrice PriceSource    Bid    Ask UnrealizedPnL  Strike Right   Expiry MarketVal  Position
DU9233079    TSM     OPT TSM 20250912 215.0 P $293.20          N/A        None $-1.00 $-1.00           N/A   215.0     P 20250912       N/A      -1.0
DU9233079    IWM     OPT IWM 20250919 260.0 C  $11.63        $0.04       Close $-1.00 $-1.00     $-1159.07   260.0     C 20250919     $4.00       1.0
DU9233079    TSM     OPT TSM 20250912 205.0 P $149.80          N/A        None $-1.00 $-1.00           N/A   205.0     P 20250912       N/A       1.0
DU9233079    IWM     OPT IWM 20250919 248.0 C $100.37        $0.65       Close $-1.00 $-1.00      $9971.66   248.0     C 20250919   $-65.00      -1.0
DU9233079    UNH     OPT UNH 20250926 350.0 C $279.34       $11.02       Close $-1.00 $-1.00     $26831.66   350.0     C 20250926 $-1102.00      -1.0
DU9233079    UNH     OPT UNH 20250926 360.0 C $187.66        $6.57       C

In [6]:
# Find and display vertical spreads
if connected and len(app.positions) > 0:
    spreads_df = app.find_vertical_spreads(positions_df)
    
    if not spreads_df.empty:
        print(f"\nVertical Spreads ({len(spreads_df)} found):")
        print("=" * 100)
        
        display_spreads = spreads_df[columns_order].copy()
        
        for col in numeric_cols:
            display_spreads[col] = display_spreads[col].apply(lambda x: f"${x:.2f}" if pd.notnull(x) else "N/A")
        
        print(display_spreads.to_string(index=False))
    else:
        print("\nNo vertical spreads found in current positions")


Vertical Spreads (3 found):
  Account Symbol SecType                    Description  AvgCost CurrentPrice PriceSource Bid Ask UnrealizedPnL      Strike Right   Expiry MarketVal  Position
DU9233079    IWM  SPREAD Bear Call 248.0/260.0 20250919  $-88.74       $-0.61  Calculated N/A N/A      $8812.59 248.0/260.0     C 20250919   $-61.00       1.0
DU9233079    TSM  SPREAD  Bull Put 205.0/215.0 20250912 $-143.39          N/A  Calculated N/A N/A           N/A 205.0/215.0     P 20250912       N/A       1.0
DU9233079    UNH  SPREAD Bear Call 350.0/360.0 20250926  $-91.68       $-4.45  Calculated N/A N/A      $8722.59 350.0/360.0     C 20250926  $-445.00       1.0


In [7]:
# Summary statistics
if connected and len(app.positions) > 0:
    print("\nPosition Summary:")
    print("=" * 50)
    
    total_positions = len(positions_df)
    option_positions = len(positions_df[positions_df['SecType'] == 'OPT'])
    stock_positions = len(positions_df[positions_df['SecType'] == 'STK'])
    spread_positions = len(spreads_df) if not spreads_df.empty else 0
    
    print(f"Total Positions: {total_positions}")
    print(f"Stock Positions: {stock_positions}")
    print(f"Option Positions: {option_positions}")
    print(f"Vertical Spreads: {spread_positions}")
    
    valid_pnl = positions_df[pd.notnull(positions_df['UnrealizedPnL'])]['UnrealizedPnL']
    if not valid_pnl.empty:
        total_pnl = valid_pnl.sum()
        print(f"\nTotal Unrealized P&L (positions with pricing): ${total_pnl:.2f}")
    
    print(f"\nData retrieved at: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Position Summary:
Total Positions: 6
Stock Positions: 0
Option Positions: 6
Vertical Spreads: 3

Total Unrealized P&L (positions with pricing): $17535.18

Data retrieved at: 2025-09-12 08:48:26


In [8]:
# Cleanup - disconnect from TWS
if connected:
    app.disconnect_tws()
    print("\nNow disconected from IBKR")

INFO:ibapi.client:disconnecting
INFO:ibapi.wrapper:ANSWER connectionClosed {}
INFO:__main__:Disconnected from TWS/Gateway



Now disconected from IBKR


## Database Integration

The following sections connect to the PostgreSQL database to retrieve trade ideas and match them with actual IBKR positions for performance analysis.

### Load Database Configuration

Setting up paths and loading PostgreSQL credentials from the configuration file.

In [11]:
# Database Connection Setup and Configuration
import sys
import os
import psycopg2
import warnings
import json
import re
from datetime import datetime

# Suppress pandas warnings
warnings.filterwarnings('ignore', message='pandas only supports SQLAlchemy connectable')

# Setup paths for database connection
notebook_dir = os.getcwd()
# If we're in the optcom directory, this is the project root
project_root = notebook_dir if 'optcom' in notebook_dir else os.path.dirname(notebook_dir)
config_path = os.path.join(project_root, 'config')
database_path = os.path.join(project_root, 'database')

# Add to Python path
sys.path.insert(0, config_path)
sys.path.insert(0, database_path)

print(f"Project root: {project_root}")
print(f"Config path: {config_path}")

# Load PostgreSQL credentials from JSON file
credentials_file = os.path.join(config_path, 'credentials.json')
if os.path.exists(credentials_file):
    with open(credentials_file, 'r') as f:
        creds = json.load(f)
    
    pg_creds = creds['database']['postgresql']
    print("✅ Database credentials loaded successfully")
else:
    print(f"❌ Credentials file not found at: {credentials_file}")
    print("Please ensure the credentials.json file exists in the config directory")
    pg_creds = None  # Set to None if credentials can't be loaded

Project root: /home/cdodd/optcom
Config path: /home/cdodd/optcom/config
✅ Database credentials loaded successfully


### Establish Database Connection

Connecting to PostgreSQL using either the database connection class or direct psycopg2 connection.

In [12]:
# Establish Database Connection
if pg_creds is not None:
    # Set environment variables BEFORE importing database_config
    os.environ.update({
        'DB_TYPE': 'postgresql',
        'DB_HOST': pg_creds['host'],
        'DB_PORT': str(pg_creds['port']),
        'DB_NAME': pg_creds['database'],
        'DB_USER': pg_creds['user'],
        'DB_PASSWORD': pg_creds['password']
    })

    try:
        from database_config import DatabaseConfig, DatabaseConnection

        # Force create PostgreSQL connection
        config = DatabaseConfig()
        config.db_type = 'postgresql'
        config.pg_config = {
            'host': pg_creds['host'],
            'port': pg_creds['port'],
            'database': pg_creds['database'],
            'user': pg_creds['user'],
            'password': pg_creds['password']
        }

        db = DatabaseConnection(config)
        print(f"✅ Connected to PostgreSQL: {db.config.pg_config['host']}")
        print(f"📊 Database type: {db.config.db_type}")
    except Exception as e:
        print(f"❌ Database connection failed: {e}")
        print("Falling back to direct psycopg2 connection...")
        db = None
else:
    print("⚠️ Database credentials not available, skipping database connection")
    db = None

✅ Connected to PostgreSQL: 35.204.11.121
📊 Database type: postgresql


### Query Database for Option Strategies

Retrieving option strategies from the PostgreSQL database to match with IBKR positions.

In [13]:
# Query Database for Option Strategies
def get_option_strategies():
    """Get option strategies from the database"""
    if db is not None:
        # Use database connection object
        query = """
        SELECT id, strategy_type, ticker, trigger_price, strike_buy, strike_sell, 
               estimated_premium, options_expiry_date, scrape_date, strategy_status
        FROM option_strategies 
        WHERE ticker IS NOT NULL AND ticker != 'None' 
        ORDER BY scrape_date DESC, id ASC
        """
        try:
            return db.execute_query_df(query)
        except Exception as e:
            print(f"❌ Database query failed: {e}")
            return pd.DataFrame()
    else:
        # Use direct psycopg2 connection
        try:
            conn = psycopg2.connect(
                host=pg_creds['host'],
                port=pg_creds['port'],
                database=pg_creds['database'],
                user=pg_creds['user'],
                password=pg_creds['password']
            )
            
            query = """
            SELECT id, strategy_type, ticker, trigger_price, strike_buy, strike_sell, 
                   estimated_premium, options_expiry_date, scrape_date, strategy_status
            FROM option_strategies 
            WHERE ticker IS NOT NULL AND ticker != 'None' 
            ORDER BY scrape_date DESC, id ASC
            """
            
            db_strategies_df = pd.read_sql_query(query, conn)
            conn.close()
            print(f"✅ Retrieved {len(db_strategies_df)} strategies from database")
            return db_strategies_df
            
        except Exception as e:
            print(f"❌ Direct database query failed: {e}")
            return pd.DataFrame()

# Get strategies from database
db_strategies_df = get_option_strategies()

if not db_strategies_df.empty:
    print(f"\n📊 Database Strategies Sample:")
    print("=" * 80)
    print(db_strategies_df.head().to_string(index=False))
else:
    print("❌ No strategies retrieved from database")


📊 Database Strategies Sample:
 id strategy_type ticker trigger_price  strike_buy  strike_sell  estimated_premium options_expiry_date                scrape_date          strategy_status
679      Bull Put    WMT         98.28        85.0         95.0               85.0          2025-10-02 2025-09-11T11:03:42.140784                     None
678      Bull Put   AAPL        222.21       205.0        215.0              160.0          2025-10-02 2025-09-11T11:03:42.055533                     None
677      Bull Put   AAPL        221.37       205.0        215.0              130.0          2025-09-26 2025-09-11T11:03:28.548781                     None
676     Bear Call    UNH        356.38       390.0        380.0              195.0          2025-10-02 2025-09-11T11:03:04.208811 missing contract details
675     Bear Call   META        779.77       850.0        840.0              120.0          2025-09-26 2025-09-11T11:02:57.434907                     None


### Parse IBKR Spread Descriptions

Parsing IBKR spread descriptions to extract strategy type, strike prices, and expiry dates for matching with database records.

In [14]:
# Parse IBKR Description and Create Join Keys
def parse_ibkr_description(description):
    """
    Parse IBKR Description to extract strategy components
    Example: "Bear Call 248.0/260.0 20250919" -> strategy_type="Bear Call", strikes="248.0/260.0", expiry="20250919"
    """
    try:
        # Split description into parts
        parts = description.strip().split()
        
        if len(parts) < 3:
            return None, None, None, None
        
        # Extract strategy type (first two words)
        strategy_type = f"{parts[0]} {parts[1]}"
        
        # Find strike prices (pattern: number/number)
        strikes_pattern = r'(\d+\.?\d*)/(\d+\.?\d*)'
        strikes_match = re.search(strikes_pattern, description)
        
        if not strikes_match:
            return strategy_type, None, None, None
            
        strike_1 = float(strikes_match.group(1))
        strike_2 = float(strikes_match.group(2))
        
        # Find expiry date (YYYYMMDD pattern)
        expiry_pattern = r'(\d{8})'
        expiry_match = re.search(expiry_pattern, description)
        
        if not expiry_match:
            return strategy_type, strike_1, strike_2, None
            
        expiry_yyyymmdd = expiry_match.group(1)
        
        # Convert YYYYMMDD to YYYY-MM-DD
        try:
            expiry_date = datetime.strptime(expiry_yyyymmdd, '%Y%m%d').date()
            expiry_formatted = expiry_date.strftime('%Y-%m-%d')
        except:
            expiry_formatted = None
        
        return strategy_type, strike_1, strike_2, expiry_formatted
        
    except Exception as e:
        print(f"Error parsing description '{description}': {e}")
        return None, None, None, None

# Parse IBKR spreads and add join keys
if connected and not spreads_df.empty:
    print(f"\n🔍 Parsing IBKR Spread Descriptions:")
    print("=" * 60)
    
    # Add parsed columns to spreads_df
    spreads_df['parsed_strategy_type'] = None
    spreads_df['parsed_strike_1'] = None
    spreads_df['parsed_strike_2'] = None
    spreads_df['parsed_expiry'] = None
    spreads_df['db_strike_sell'] = None
    spreads_df['db_strike_buy'] = None
    
    for idx, row in spreads_df.iterrows():
        strategy_type, strike_1, strike_2, expiry = parse_ibkr_description(row['Description'])
        
        spreads_df.loc[idx, 'parsed_strategy_type'] = strategy_type
        spreads_df.loc[idx, 'parsed_strike_1'] = strike_1
        spreads_df.loc[idx, 'parsed_strike_2'] = strike_2
        spreads_df.loc[idx, 'parsed_expiry'] = expiry
        
        # Determine which strike is sell vs buy based on strategy type
        if strategy_type and strike_1 is not None and strike_2 is not None:
            if 'Bear Call' in strategy_type:
                # Bear Call: sell lower strike, buy higher strike
                spreads_df.loc[idx, 'db_strike_sell'] = min(strike_1, strike_2)
                spreads_df.loc[idx, 'db_strike_buy'] = max(strike_1, strike_2)
            elif 'Bull Put' in strategy_type:
                # Bull Put: sell higher strike, buy lower strike  
                spreads_df.loc[idx, 'db_strike_sell'] = max(strike_1, strike_2)
                spreads_df.loc[idx, 'db_strike_buy'] = min(strike_1, strike_2)
            else:
                # Default: first strike is sell, second is buy
                spreads_df.loc[idx, 'db_strike_sell'] = strike_1
                spreads_df.loc[idx, 'db_strike_buy'] = strike_2
        
        print(f"{row['Symbol']}: {row['Description']} -> {strategy_type}, {strike_1}/{strike_2}, {expiry}")
    
    print(f"\n✅ Parsed {len(spreads_df)} IBKR spreads")
else:
    print("⚠️ No IBKR spreads available to parse")


🔍 Parsing IBKR Spread Descriptions:
IWM: Bear Call 248.0/260.0 20250919 -> Bear Call, 248.0/260.0, 2025-09-19
TSM: Bull Put 205.0/215.0 20250912 -> Bull Put, 205.0/215.0, 2025-09-12
UNH: Bear Call 350.0/360.0 20250926 -> Bear Call, 350.0/360.0, 2025-09-26

✅ Parsed 3 IBKR spreads


### Join IBKR Spreads with Database Strategies

Matching IBKR spreads with database strategies based on symbol, strategy type, strike prices, and expiry dates.

In [15]:
# Join IBKR Spreads with Database Strategies
def join_spreads_with_database():
    """
    Join IBKR spreads with database strategies based on:
    - Symbol (IBKR) -> ticker (DB)
    - Strategy type (first two words from Description)
    - Strike prices (parsed from Description)
    - Expiry date (YYYYMMDD -> YYYY-MM-DD)
    """
    if spreads_df.empty or db_strategies_df.empty:
        print("⚠️ No data to join - either spreads or database strategies are empty")
        return pd.DataFrame()
    
    joined_data = []
    
    print(f"\n🔗 Attempting to join {len(spreads_df)} IBKR spreads with {len(db_strategies_df)} database strategies:")
    print("=" * 90)
    
    for ibkr_idx, ibkr_row in spreads_df.iterrows():
        symbol = ibkr_row['Symbol']
        strategy_type = ibkr_row['parsed_strategy_type']
        expiry = ibkr_row['parsed_expiry']
        db_strike_sell = ibkr_row['db_strike_sell']
        db_strike_buy = ibkr_row['db_strike_buy']
        
        print(f"\n🔍 Looking for matches for IBKR spread:")
        print(f"   Symbol: {symbol}, Strategy: {strategy_type}, Expiry: {expiry}")
        print(f"   Strike Sell: {db_strike_sell}, Strike Buy: {db_strike_buy}")
        
        # Find matching strategies in database
        matches = db_strategies_df[
            (db_strategies_df['ticker'] == symbol) &
            (db_strategies_df['strategy_type'] == strategy_type) &
            (db_strategies_df['options_expiry_date'].astype(str) == expiry) &
            (db_strategies_df['strike_sell'] == db_strike_sell) &
            (db_strategies_df['strike_buy'] == db_strike_buy)
        ]
        
        if len(matches) > 0:
            print(f"   ✅ Found {len(matches)} matching database record(s)")
            
            for db_idx, db_row in matches.iterrows():
                # Combine IBKR and DB data
                joined_record = {
                    # IBKR data
                    'ibkr_symbol': ibkr_row['Symbol'],
                    'ibkr_description': ibkr_row['Description'],
                    'ibkr_avg_cost': ibkr_row['AvgCost'],
                    'ibkr_current_price': ibkr_row['CurrentPrice'],
                    'ibkr_unrealized_pnl': ibkr_row['UnrealizedPnL'],
                    'ibkr_market_val': ibkr_row['MarketVal'],
                    'ibkr_position': ibkr_row['Position'],
                    
                    # Database data
                    'db_id': db_row['id'],
                    'db_ticker': db_row['ticker'],
                    'db_strategy_type': db_row['strategy_type'],
                    'db_trigger_price': db_row['trigger_price'],
                    'db_strike_sell': db_row['strike_sell'],
                    'db_strike_buy': db_row['strike_buy'],
                    'db_estimated_premium': db_row['estimated_premium'],
                    'db_options_expiry_date': db_row['options_expiry_date'],
                    'db_scrape_date': db_row['scrape_date'],
                    'db_strategy_status': db_row['strategy_status'],
                    
                    # Comparison fields
                    'premium_difference': (ibkr_row['AvgCost'] - db_row['estimated_premium']) if pd.notnull(ibkr_row['AvgCost']) and pd.notnull(db_row['estimated_premium']) else None,
                }
                
                joined_data.append(joined_record)
                
                print(f"      DB ID: {db_row['id']}, Estimated Premium: ${db_row['estimated_premium']}, Status: {db_row['strategy_status']}")
        else:
            print(f"   ❌ No matching database records found")
    
    if joined_data:
        joined_df = pd.DataFrame(joined_data)
        print(f"\n✅ Successfully joined {len(joined_df)} records")
        return joined_df
    else:
        print(f"\n⚠️ No matches found between IBKR spreads and database strategies")
        return pd.DataFrame()

# Perform the join
if connected and not spreads_df.empty and not db_strategies_df.empty:
    joined_df = join_spreads_with_database()
else:
    joined_df = pd.DataFrame()
    print("⚠️ Skipping join - missing required data")


🔗 Attempting to join 3 IBKR spreads with 504 database strategies:

🔍 Looking for matches for IBKR spread:
   Symbol: IWM, Strategy: Bear Call, Expiry: 2025-09-19
   Strike Sell: 248.0, Strike Buy: 260.0
   ✅ Found 1 matching database record(s)
      DB ID: 652, Estimated Premium: $88.0, Status: order placed

🔍 Looking for matches for IBKR spread:
   Symbol: TSM, Strategy: Bull Put, Expiry: 2025-09-12
   Strike Sell: 215.0, Strike Buy: 205.0
   ✅ Found 1 matching database record(s)
      DB ID: 551, Estimated Premium: $143.0, Status: order placed

🔍 Looking for matches for IBKR spread:
   Symbol: UNH, Strategy: Bear Call, Expiry: 2025-09-26
   Strike Sell: 350.0, Strike Buy: 360.0
   ✅ Found 1 matching database record(s)
      DB ID: 662, Estimated Premium: $90.0, Status: order placed

✅ Successfully joined 3 records


### Display Matched Results

Comprehensive analysis showing IBKR spreads matched with database strategies, including premium differences and performance metrics.

In [16]:
# Display Joined Results
if not joined_df.empty:
    print(f"\n📊 IBKR Spreads Joined with Database Strategies ({len(joined_df)} matches):")
    print("=" * 120)
    
    # Create a clean display dataframe
    display_columns = [
        'ibkr_symbol', 'db_strategy_type', 'ibkr_description',
        'ibkr_avg_cost', 'db_estimated_premium', 'premium_difference',
        'ibkr_current_price', 'ibkr_unrealized_pnl', 
        'db_strategy_status', 'db_scrape_date'
    ]
    
    display_df = joined_df[display_columns].copy()
    
    # Format numeric columns
    numeric_cols = ['ibkr_avg_cost', 'db_estimated_premium', 'premium_difference', 
                   'ibkr_current_price', 'ibkr_unrealized_pnl']
    
    for col in numeric_cols:
        if col in display_df.columns:
            display_df[col] = display_df[col].apply(
                lambda x: f"${x:.2f}" if pd.notnull(x) else "N/A"
            )
    
    # Format dates
    display_df['db_scrape_date'] = display_df['db_scrape_date'].apply(
        lambda x: str(x)[:10] if pd.notnull(x) else "N/A"
    )
    
    # Rename columns for better display
    display_df = display_df.rename(columns={
        'ibkr_symbol': 'Symbol',
        'db_strategy_type': 'Strategy',
        'ibkr_description': 'IBKR Description', 
        'ibkr_avg_cost': 'IBKR Avg Cost',
        'db_estimated_premium': 'DB Est. Premium',
        'premium_difference': 'Premium Diff',
        'ibkr_current_price': 'Current Price',
        'ibkr_unrealized_pnl': 'Unrealized P&L',
        'db_strategy_status': 'DB Status',
        'db_scrape_date': 'DB Scrape Date'
    })
    
    print(display_df.to_string(index=False))
    
    # Summary statistics
    if len(joined_df) > 0:
        print(f"\n📈 Summary:")
        print("=" * 50)
        print(f"Total matched positions: {len(joined_df)}")
        
        # Premium difference analysis
        valid_diffs = joined_df.dropna(subset=['premium_difference'])
        if len(valid_diffs) > 0:
            avg_diff = valid_diffs['premium_difference'].mean()
            print(f"Average premium difference: ${avg_diff:.2f}")
            print(f"(Positive = IBKR cost more than DB estimate)")
            
        # Status breakdown
        status_counts = joined_df['db_strategy_status'].value_counts()
        print(f"\nDatabase status breakdown:")
        for status, count in status_counts.items():
            print(f"  {status}: {count}")

else:
    print("\n⚠️ No joined data available to display")
    
    # Show what we have separately
    if connected and not spreads_df.empty:
        print(f"\n📊 Available IBKR Spreads ({len(spreads_df)}):")
        for _, row in spreads_df.iterrows():
            print(f"  {row['Symbol']}: {row['Description']}")
    
    if not db_strategies_df.empty:
        print(f"\n📊 Available Database Strategies (sample of {min(5, len(db_strategies_df))}):")
        for _, row in db_strategies_df.head().iterrows():
            print(f"  {row['ticker']}: {row['strategy_type']} {row['strike_sell']}/{row['strike_buy']} {row['options_expiry_date']}")

print(f"\n✅ Analysis completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


📊 IBKR Spreads Joined with Database Strategies (3 matches):
Symbol  Strategy               IBKR Description IBKR Avg Cost DB Est. Premium Premium Diff Current Price Unrealized P&L    DB Status DB Scrape Date
   IWM Bear Call Bear Call 248.0/260.0 20250919       $-88.74          $88.00     $-176.74        $-0.61       $8812.59 order placed     2025-09-05
   TSM  Bull Put  Bull Put 205.0/215.0 20250912      $-143.39         $143.00     $-286.39           N/A            N/A order placed     2025-08-20
   UNH Bear Call Bear Call 350.0/360.0 20250926       $-91.68          $90.00     $-181.68        $-4.45       $8722.59 order placed     2025-09-09

📈 Summary:
Total matched positions: 3
Average premium difference: $-214.94
(Positive = IBKR cost more than DB estimate)

Database status breakdown:
  order placed: 3

✅ Analysis completed at: 2025-09-12 08:59:32
